In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests
import json
from pprint import pprint
from datetime import datetime
from pathlib import Path

In [2]:
# #csv sports_io data pull if the api is not utilized

# data_dict = {}
# csv_path = f'../data/sports_io/'

# for i in os.listdir(csv_path):
#     data_df = pd.read_csv(csv_path + i)
    
#     data_dict[i.split('.')[0]] = data_df

# print(data_dict.keys())

In [20]:
#sports_io nba documentation link:  https://fantasydata.com/api/api-documentation/nba#
#data_set_list = ['GamesByDate', 'GameOddsByDate', 'GameOddsLineMovement', 'Schedules', 'Stadiums', 'TeamGameStatsByDate', 'TeamSeasonStats']
#odds_data_set_dict = {'GamesByDate' : 'YYYY-MM-DD', 'GameOddsByDate' : 'YYYY-MM-DD', 'GameOddsLineMovement' : 'GameID', 'Schedules' : 'Season', 'Stadiums' : NaN, 'TeamGameStatsByDate' : 'YYYY-MM-DD', 'TeamSeasonStats' : 'Season'}

#assign sports_io api key
load_dotenv()
sports_io_api_key = os.getenv('sports_io_api_key')


#assign sports_io url to variable that can be pulled with different parameters
def sports_io_api_pull(data_set, data_set_parameter, sports_io_api_key):
    sports_io_odds_url = f"https://api.sportsdata.io/api/nba/odds/json/{data_set}/{data_set_parameter}?key={sports_io_api_key}"

    response_data = requests.get(sports_io_odds_url).json()

    response_df = pd.DataFrame(response_data)

    return response_df

In [18]:
date_list = pd.date_range(start = '1/2/2020', periods = 5).strftime(('%Y-%m-%d'))
date_list

Index(['2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06'], dtype='object')

In [53]:
data_set = 'GamesByDate'
date_list = pd.date_range(start = '1/2/2020', periods = 5).strftime(('%Y-%m-%d'))

nba_odds_list = []

for i in date_list:

    nba_odds_data = sports_io_api_pull(data_set, i, sports_io_api_key)
    nba_odds_list.append(nba_odds_data)

nba_odds_df = pd.concat(nba_odds_list, axis = 0)

In [54]:
nba_odds_df_clean = nba_odds_df.drop(columns = 'Quarters')

nba_odds_df_clean.head()

,GameID,Season,SeasonType,Status,Day,DateTime,AwayTeam,HomeTeam,AwayTeamID,HomeTeamID,StadiumID,AwayTeamScore,HomeTeamScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine,HomeRotationNumber,AwayRotationNumber,NeutralVenue
0,14411,2020,1,Final,2020-01-02T00:00:00,2020-01-02T19:00:00,CHA,CLE,2,12,12,109,106,-1.5,212.5,105,-125,546.0,545.0,False
1,14412,2020,1,Final,2020-01-02T00:00:00,2020-01-02T19:00:00,DEN,IND,20,13,13,124,116,1.0,208.5,-113,-107,548.0,547.0,False
2,14413,2020,1,Final,2020-01-02T00:00:00,2020-01-02T19:30:00,TOR,MIA,10,4,4,76,84,-5.5,214.5,177,-217,550.0,549.0,False
3,14414,2020,1,Final,2020-01-02T00:00:00,2020-01-02T20:00:00,UTA,CHI,17,11,11,102,98,3.0,209.0,-155,130,NaN,NaN,False
4,14415,2020,1,Final,2020-01-02T00:00:00,2020-01-02T20:00:00,GS,MIN,26,16,16,84,99,-3.0,212.0,126,-150,552.0,551.0,False


In [55]:
nba_odds_df_clean.columns

Index(['GameID', 'Season', 'SeasonType', 'Status', 'Day', 'DateTime',
       'AwayTeam', 'HomeTeam', 'AwayTeamID', 'HomeTeamID', 'StadiumID',
       'AwayTeamScore', 'HomeTeamScore', 'PointSpread', 'OverUnder',
       'AwayTeamMoneyLine', 'HomeTeamMoneyLine', 'HomeRotationNumber',
       'AwayRotationNumber', 'NeutralVenue'],
      dtype='object')

In [159]:
data_set_ii = 'GameOddsLineMovement'
game_list = list(nba_odds_df_clean['GameID'])

opening_lines_list = []

for i in game_list:

    line_movement_data = sports_io_api_pull(data_set_ii, i, sports_io_api_key)

    line_movement_data['PregameOdds'] = line_movement_data['PregameOdds'].apply(pd.DataFrame)
    line_movement_data['opening_lines'] = line_movement_data['PregameOdds'].apply(lambda x: x.loc[x['Created'] == x['Created'].min(), 'HomePointSpread'])
    line_movement_data = line_movement_df[['GameId', 'opening_lines']]

    opening_lines_list.appending(line_movement_data)

opening_lines_df = opening_lines_df

,GameId,opening_lines
0,14411,-2.5


In [154]:
line_movement_df['PregameOdds'] = line_movement_df['PregameOdds'].apply(pd.DataFrame)
line_movement_df['opening_lines'] = line_movement_df['PregameOdds'].apply(lambda x: x.loc[x['Created'] == x['Created'].min(), 'HomePointSpread'])
# line_movement_df['opening_line'] = line_movement_df['PregameOdds'].apply(lambda x: x.loc[x['Created'] == x['Created'].min(), x['HomePointSpread']])

# line_movement_df['opening_line'] = line_movement_df['PregameOdds'].apply(lambda x: x.loc[x['Created'] == x['Created'].min(), :])


In [157]:
line_movement_df['opening_lines'] = line_movement_df['PregameOdds'].apply(lambda x: x.loc[x['Created'] == x['Created'].min(), 'HomePointSpread'])

In [158]:
line_movement_df.head()

,GameId,Season,SeasonType,Day,DateTime,Status,AwayTeamId,HomeTeamId,AwayTeamName,HomeTeamName,HomeTeamScore,AwayTeamScore,TotalScore,HomeRotationNumber,AwayRotationNumber,PregameOdds,opening_lines
0,14411,2020,1,2020-01-02T00:00:00,2020-01-02T19:00:00,Final,2,12,CHA,CLE,106,109,215,546,545,GameOddId Sportsbook GameId ...,-2.5


In [145]:
test = pd.concat(list(line_movement_df['PregameOdds']))

In [152]:
test = line_movement_df['PregameOdds'][0]
test = test.loc[test['Created'] == test['Created'].min(), 'HomePointSpread']
test

38   -2.5
Name: HomePointSpread, dtype: float64

In [107]:
test = line_movement_df.loc[0, 'PregameOdds']

test.head()

,GameOddId,Sportsbook,GameId,Created,Updated,HomeMoneyLine,AwayMoneyLine,HomePointSpread,AwayPointSpread,HomePointSpreadPayout,AwayPointSpreadPayout,OverUnder,OverPayout,UnderPayout,SportsbookId
0,1502546,None,14411,2020-01-02T18:02:24,2020-01-02T18:59:52,-130.0,110.0,-2.0,2.0,-105,-105,212.0,-110,-110,None
1,1502542,None,14411,2020-01-02T17:57:14,2020-01-02T18:01:50,-125.0,105.0,-2.0,2.0,-105,-105,212.0,-110,-110,None
2,1502525,None,14411,2020-01-02T17:54:13,2020-01-02T17:56:13,-125.0,105.0,-1.5,1.5,-110,100,212.0,-110,-110,None
3,1502522,None,14411,2020-01-02T17:52:18,2020-01-02T17:53:54,-125.0,105.0,-1.5,1.5,-110,100,211.5,-110,-110,None
4,1502521,None,14411,2020-01-02T17:48:46,2020-01-02T17:51:45,-125.0,105.0,-2.0,2.0,-105,-105,211.5,-110,-110,None


In [114]:
test['Created'].min()

# type(test)

pandas.core.frame.DataFrame

In [115]:
test = line_movement_df.loc[0, 'PregameOdds']

test.loc[test['Created'] == test['Created'].min(), ['HomePointSpread']]

# test.head()

,HomePointSpread
38,-2.5


In [85]:
line_movement_df['PregameOdds'] = line_movement_df['PregameOdds'].apply(pd.DataFrame)

line_movement_df.head()

,GameId,Season,SeasonType,Day,DateTime,Status,AwayTeamId,HomeTeamId,AwayTeamName,HomeTeamName,HomeTeamScore,AwayTeamScore,TotalScore,HomeRotationNumber,AwayRotationNumber,PregameOdds
0,14411,2020,1,2020-01-02T00:00:00,2020-01-02T19:00:00,Final,2,12,CHA,CLE,106,109,215,546,545,GameOddId Sportsbook GameId ...
